In [1]:
pip install googledrivedownloader

Note: you may need to restart the kernel to use updated packages.


In [2]:
from google_drive_downloader.google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1s4bVYjVwfHBPIcy6W5XAm4sjrGMjFlG_',
                                    dest_path='/content/train.zip',
                                    unzip=True)
gdd.download_file_from_google_drive(file_id='1_0Rzyx9A41WqeAja2seZEmEw2_OGKPg3',
                                    dest_path='/content/test.zip',
                                    unzip=True)

Unzipping...Done.
Unzipping...Done.


In [3]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models
import cv2 
import matplotlib.pyplot as plt


In [4]:
batch_size=100

train_data_ex = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_data_ex.flow_from_directory(
        '/content/training',  
        target_size=(64,64),
        batch_size=batch_size,
        class_mode='binary')

Found 5875 images belonging to 2 classes.


In [5]:
model = models.Sequential()
model.add(layers.Conv2D(64, kernel_size=4, strides=1, activation='relu', input_shape=(64,64,3),padding="same"))
model.add(layers.AveragePooling2D((2,2),strides=1))
model.add(layers.Conv2D(128, kernel_size=4, strides=1, activation='relu', padding="same"))
model.add(layers.AveragePooling2D((2,2),strides=1))
model.add(layers.Conv2D(256, kernel_size=4, strides=1, activation='relu', padding="same"))
model.add(layers.AveragePooling2D((2,2),strides=1))
model.add(layers.Flatten())
model.add(layers.Dense(512))
model.add(layers.Dense(1,activation="sigmoid"))
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])
'''
checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True,
    save_freq=1000)
'''

model.fit(train_generator, steps_per_epoch=10, epochs=200, batch_size=100)#,callbacks=[model_checkpoint_callback]) 

#model.load_weights(checkpoint_filepath)
from keras.models import model_from_json
json_model = model.to_json()
with open('./Tensored.json', 'w') as json_file:
    json_file.write(json_model)
model.save_weights('./Tensored_weights.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        3136      
_________________________________________________________________
average_pooling2d (AveragePo (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 63, 63, 128)       131200    
_________________________________________________________________
average_pooling2d_1 (Average (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 62, 256)       524544    
_________________________________________________________________
average_pooling2d_2 (Average (None, 61, 61, 256)       0         
_________________________________________________________________
flatten (Flatten)            (None, 952576)            0

In [6]:
import os
res={}
for i in range(1,99):
  img = image.load_img('/content/test/'+str(i)+'.jpg')
  X = image.img_to_array(img)
  X=np.expand_dims(X,axis=0)
  images =  np.vstack([X])
  value = model.predict(images)
  test_set = str(i) + '.jpg'
  res[test_set] = int(value[0][0])



In [7]:
import json
def write_json(filename, result):
    with open(filename, 'w') as outfile:
        json.dump(result, outfile)
def read_json(filename):
    with open(filename, 'r') as outfile:
        data =  json.load(outfile)
    return data

write_json('./result.json', res)
